# Rurality and shopping area 
## What happens
- The script below computes how many points that are inside a radius of `distance_meters` meters of the point.
- Computes how many stores that are closer than `distance_meters` meters from another store for all the stores
- Computes how many busstops that are closer than `distance_meters` meters from a store for all the stores
- Computes how many busstops that are closer than `distance_meters` meters from another busstop for all the busstops (not sure if this is important or not)
- Cumputes the distance to the closest busstop for all stores.

## Computational improvements
### Initial attempt
- Pure python
- Would have run in at least a couple of days to compute the pairs of distances and counts
- Not acceptable

### Optimizing with numba first attempt
- Using `njit` decorator of `numba`
- Computation down to about 10 minutes
- Still a bit slow for parameter tuning for the `distance_meters`-variable

### Parallelizing with numba
- Using `prange` and `parallel=True` from `numba` 
- Cutting the time to a couple of minutes
### Further improvements - Cuda
- Possibly much faster with GPU, but currently not prioritized

### Why not compute all pariwise distances once and store it
Whould have taken $8B*50122^2\approx 20GB$ where `50122` is the total number of stores, and 8B is the size of a float64. Could have changed to int32 which would have reduced the size to 10GB. Could have made the matrix triangular and thus reduced the size to 5GB, but it would still be to big.
This would needed to be done for `stores x stores`, `stores x busstops` and possibly `busstops x busstops`

In [ ]:
import numpy as np
import pandas as pd
from shapely import wkt
import geopandas as gpd
from numba import prange, njit
from math import radians, cos, sin, asin, sqrt  

In [ ]:
busstops = pd.read_csv('../../data/busstops_norway.csv')
stores_extra = pd.read_csv('../../data/stores_extra.csv')
stores_train = pd.read_csv('../../data/stores_train.csv')
stores_test = pd.read_csv('../../data/stores_test.csv')
grunnkrets = pd.read_csv('../../data/grunnkrets_norway_stripped.csv')

busstops['geometry'] = busstops['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(busstops, crs='EPSG:4326')

In [ ]:
grunnkrets = grunnkrets.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')


In [ ]:
# For calculating the distance in Kilometres   
@njit
def geo_distance(La1, Lo1, La2, Lo2):  
       
    # The math module contains the function name "radians" which is used for converting the degrees value into radians.  
    Lo1 = radians(Lo1)  
    Lo2 = radians(Lo2)  
    La1 = radians(La1)  
    La2 = radians(La2)  
        
    # Using the "Haversine formula"  
    D_Lo = Lo2 - Lo1  
    D_La = La2 - La1  
    P = sin(D_La / 2)**2 + cos(La1) * cos(La2) * sin(D_Lo / 2)**2  
   
    Q = 2 * asin(sqrt(P))  
      
    # The radius of earth in kilometres.  
    R_km = 6371  
        
    # calculate result in meters
    return(Q * R_km)  * 1000

In [ ]:
@njit('i4[:](f8[:,:], f8[:,:], i8, b1)', parallel=True)
def number_of_points_closer_than_numba(x, y, distance_meters: int = 1000, x_and_y_same_array=False):
    z = np.empty(x.shape[0], dtype=np.int32)
    num_rows = x.shape[0]
    for i in prange(num_rows):
        # number of points closer that distance_meters from point
        count = np.count_nonzero(np.array([geo_distance(x[i][0], x[i][1], y[j][0], y[j][1]) < distance_meters 
            for j in prange(y.shape[0])]))
        # remove 1 if x and y are equal as distance to self is 0
        if x_and_y_same_array:
            count -= 1
        z[i] = count
    return z
    

In [ ]:
@njit('Tuple((i4[:], f8[:]))(f8[:,:], f8[:,:], b1)', parallel=True)
def closest_points_numba(x, y, x_and_y_same_array=True):
    closest_points_distance = np.empty(x.shape[0], dtype=np.float64)
    closest_points_index = np.empty(x.shape[0], dtype=np.int32)

    for i in prange(x.shape[0]):
        # closest point that is not itself
        distance_points = np.array([geo_distance(x[i][0], x[i][1], y[j][0], y[j][1]) if x_and_y_same_array == False or x[i][0] != y[j][0] and x[i][1] != y[j][1] else np.inf
                                    for j in range(y.shape[0])])
        closest_point_index = np.argmin(distance_points)
        closest_point_distance = np.min(distance_points)
        closest_points_distance[i] = closest_point_distance
        closest_points_index[i] = closest_point_index

    return closest_points_index, closest_points_distance

In [ ]:
def append_number_of_points_closer_than(df, points_in_df, geo_points, column_name, distance_meters: int = 1000, x_and_y_same_array=False):
    close_elements = number_of_points_closer_than_numba(x=points_in_df, y=geo_points, distance_meters=distance_meters, x_and_y_same_array=x_and_y_same_array)
    df[column_name] = close_elements

In [ ]:
def append_closest_points(df, geo_df, points_in_df, geo_points, index_column_name, new_column_name, x_and_y_same_array=False):
    closest_elements_index, closest_elements_distance = closest_points_numba(points_in_df, geo_points, x_and_y_same_array=x_and_y_same_array)
    df[new_column_name] = closest_elements_distance
    # add id of point that was closest to each point in df 
    print(closest_elements_distance, closest_elements_index)
    df[index_column_name] = geo_df.iloc[closest_elements_index].set_index(df.index)[index_column_name]


In [ ]:
# Creating a np array of [lat, lon] from shapely GeoPoints
geo_busstops = np.array(list([geo.y, geo.x] for geo in gdf.geometry.to_numpy()))

In [ ]:
append_number_of_points_closer_than(gdf, geo_busstops, geo_busstops, "num_of_buss_stops_closer_that_1000_to_busstop", x_and_y_same_array=True)
gdf.to_csv("../../own_data/busstops_norway_with_count.csv")

In [ ]:
# concat all data of stores together, adding index with train name from where the row came from
stores = pd.concat({"train": stores_train, "extra": stores_extra, "test": stores_test})

In [ ]:
# all the lat lons of the stores, to a numpy array so that the numba-function can compute the distances
geo_stores = stores[["lat", "lon"]].to_numpy()

In [ ]:
%%time
# append number of close stores and close busstops
append_number_of_points_closer_than(stores, geo_stores, geo_stores, "other_stores_1000", x_and_y_same_array=True, distance_meters=1000)
append_number_of_points_closer_than(stores, geo_stores, geo_stores, "other_stores_100", x_and_y_same_array=True, distance_meters=100)
append_number_of_points_closer_than(stores, geo_stores, geo_stores, "other_stores_50", x_and_y_same_array=True, distance_meters=50)
append_number_of_points_closer_than(stores, geo_stores, geo_stores, "other_stores_250", x_and_y_same_array=True, distance_meters=250)
append_number_of_points_closer_than(stores, geo_stores, geo_busstops, "buss_stops_1000", x_and_y_same_array=False, distance_meters=1000)
append_number_of_points_closer_than(stores, geo_stores, geo_busstops, "buss_stops_300", x_and_y_same_array=False, distance_meters=300)

CPU times: user 14min 43s, sys: 136 ms, total: 14min 43s
Wall time: 3min 49s


In [ ]:
append_closest_points(
    df=stores, 
    geo_df=busstops, 
    points_in_df=geo_stores, 
    geo_points=geo_busstops, 
    index_column_name='busstop_id', 
    new_column_name="distance_closest_busstop", 
    x_and_y_same_array=False)

[495.65295876 102.35551582  25.53176743 ... 114.20099103  63.82726397
 125.76760068] [48753   140 58056 ... 30708  9534 56169]


In [ ]:
# find all stores that has a grunnkrets_id that doesn't exist in the grunnkrets table
stores_wo_grunnkrets = stores.merge(grunnkrets[['grunnkrets_id', 'grunnkrets_name']], how="left", on="grunnkrets_id")
nan_stores_all = stores_wo_grunnkrets.loc[stores_wo_grunnkrets['grunnkrets_name'].isna()]
outer_stores_test = stores_test.merge(grunnkrets[['grunnkrets_id', 'grunnkrets_name']], how="left", on="grunnkrets_id")
nan_stores = outer_stores_test.loc[outer_stores_test['grunnkrets_name'].isna()]

In [ ]:
# copy grunnkrets of stores so that we can impute the grunnkrets_id of the test_stores that has a bad grunnkrets_id
stores['grunnkrets_1'] = stores['grunnkrets_id']
append_closest_points(
    df=nan_stores,
    geo_df=stores[~stores.grunnkrets_id.isin(nan_stores_all.grunnkrets_id)],
    points_in_df=nan_stores[["lat", "lon"]].to_numpy(),
    geo_points=geo_stores,
    index_column_name='grunnkrets_1',
    new_column_name='closest_store_id',
    x_and_y_same_array=True
)

[8.33037557e+02 1.13153469e+02 8.71270649e-01 9.92971322e+03
 6.35282105e+03 2.77653490e+04 1.59262209e+03 1.51000246e+04
 6.76384356e+03 2.38863345e+02 2.93062827e+03 1.56463351e+03
 6.54021281e+03 2.73123562e+04 4.19203250e+02 2.04058526e+04
 2.35162647e+04 2.46535005e+03 1.41086385e+03 1.13350247e+04
 1.76739494e+03 4.45915791e+03 1.26082022e+04 3.61803004e+03
 1.68662920e+03 8.14563288e+03 2.93062827e+03] [47589 16979 14155  1596  7099 46512 49150 14062 28820  7907 49856 14595
 16959 10586  1375 11272  2199 14393 14714  9886 13520 25574 46335   659
 42344 14702 44173]


/tmp/ipykernel_624097/1070740115.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name] = closest_elements_distance
/tmp/ipykernel_624097/1070740115.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[index_column_name] = geo_df.iloc[closest_elements_index].set_index(df.index)[index_column_name]


In [ ]:
# rename column of the grunnkrets_id of the stores with bad grunnkrets_id
nan_stores.grunnkrets_id = nan_stores.grunnkrets_1


/tmp/ipykernel_624097/1378478374.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_stores.grunnkrets_id = nan_stores.grunnkrets_1


In [ ]:
# set grunnkrets_id to grunnkrets_id of the closest store, that is a good grunnkrets_id
for _, row in nan_stores[['store_id', 'grunnkrets_1']].iterrows():
    stores.loc[stores.store_id == row['store_id'], 'grunnkrets_id'] = row.grunnkrets_1

In [ ]:
stores.drop(columns="grunnkrets_1")
stores.to_csv("../../own_data/stores.csv")

In [ ]:
stores = pd.read_csv("../../own_data/stores.csv")
busstops = pd.read_csv("../../own_data/busstops_norway_with_count.csv")
age_dist = pd.read_csv("../../data/grunnkrets_age_distribution.csv")
households = pd.read_csv("../../data/grunnkrets_households_num_persons.csv")
income = pd.read_csv("../../data/grunnkrets_income_households.csv")
grunnkrets = pd.read_csv("../../data/grunnkrets_norway_stripped.csv")
plaace = pd.read_csv("../../data/plaace_hierarchy.csv")



In [ ]:
grunnkrets.drop(columns=['grunnkrets_name', 'geometry'], inplace=True)

In [ ]:
grunnkrets = grunnkrets.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')

In [ ]:
# remove 2015 data and year column
households = households.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')
income = income.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')
age_dist = age_dist.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')
grunnkrets.drop(columns='year', inplace=True)
households.drop(columns='year', inplace=True)
income.drop(columns='year', inplace=True)
age_dist.drop(columns='year', inplace=True)

In [ ]:
# set index to grunnkrets_id for merging
households.set_index('grunnkrets_id', inplace=True)
grunnkrets.set_index('grunnkrets_id', inplace=True)
income.set_index('grunnkrets_id', inplace=True)
age_dist.set_index('grunnkrets_id', inplace=True)

In [ ]:
busstops.drop(columns=['geometry', 'Unnamed: 0'], inplace=True)

In [ ]:
stores = stores.rename(columns={"Unnamed: 0": "dataset", "Unnamed: 1": "range_index"}).set_index(["dataset", "range_index"])
stores.drop(columns='sales_channel_name', inplace=True)
stores['copy_index'] = stores.index


In [ ]:
all_merged = stores.merge(grunnkrets, left_on='grunnkrets_id', right_index=True) \
                   .merge(households, left_on='grunnkrets_id', right_index=True, how="left") \
                   .merge(income, left_on='grunnkrets_id', right_index=True, how="left") \
                   .merge(age_dist, left_on='grunnkrets_id', right_index=True, how="left") \
                   .merge(plaace, on='plaace_hierarchy_id') \
                   .merge(busstops, on='busstop_id')

In [ ]:
all_merged.index = pd.MultiIndex.from_tuples(
    all_merged['copy_index'], 
    names=['dataset', 'range_index'])

all_merged.drop(columns=['copy_index', 'year', 'plaace_hierarchy_id', 'sales_channel_name', 'lv1_desc', 'lv2_desc', 'lv3_desc', 'lv4_desc'], inplace=True)

In [ ]:
all_merged.to_csv("../../own_data/all_merged.csv")

## What happens here?
- The grunnkrets and age distribution data is loaded.
- 2016-values are kept if they exist
- Total population and age groups 0-14, 15-64, 64-90 is aggregated
- Population density is computed

## Remarks
- There are 1891 NaNs, around 15%, because we miss age data for some grunnkrets'.

## Further improvements
- Outlier detection: Small areas yield large densities. Small areas and small populations might be set to NaNs.

In [ ]:
grunnkrets = pd.read_csv('../../data/grunnkrets_norway_stripped.csv')
grunnkrets_ages = pd.read_csv('../../data/grunnkrets_age_distribution.csv')
grunnkrets_household = pd.read_csv("../../data/grunnkrets_households_num_persons.csv")
grunnkrets_household = grunnkrets_household.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')
grunnkrets_ages = grunnkrets_ages.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')
grunnkrets = grunnkrets.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')

In [ ]:
grunnkrets_household.drop(columns="year", inplace=True)
grunnkrets_household["sum_people"] = grunnkrets_household.drop(columns="grunnkrets_id").sum(axis=1)

In [ ]:
df = grunnkrets_ages.drop_duplicates(subset=["grunnkrets_id"], keep='last')
df = grunnkrets.merge(df, how='left', on='grunnkrets_id')
df["grunnkrets_population"] = df.drop(columns=["grunnkrets_id", "year_x", "year_y", "area_km2", "geometry", "municipality_name", "district_name", "grunnkrets_name"]).sum(axis=1)
df = df.merge(grunnkrets_household[['grunnkrets_id', "sum_people"]], how="left", on="grunnkrets_id")
df['grunnkrets_population'] = df.apply(
    lambda row: row['sum_people'] if row['grunnkrets_population'] == 0 else row['grunnkrets_population'],
    axis=1
)
df["district_population"] = df.groupby("district_name")["grunnkrets_population"].transform("sum")
df["municipality_population"] = df.groupby("municipality_name")["grunnkrets_population"].transform("sum")
df["district_area"] = df.groupby("district_name")["area_km2"].transform("sum")
df["municipality_area"] = df.groupby("municipality_name")["area_km2"].transform("sum")
ages = [0,3,7,13,18,25,31,41,54,65,78,91]
# ages = [0,5,12,18,30,45,65,91]
ages = [0,15,35,65,91]
# ages = list(range(92))
for j in range(1, len(ages)):  
    age_from = ages[j-1]
    age_to = ages[j]
    df[f"grunnkrets_age_{age_from}-{age_to-1}"] = df[df.columns[df.columns.isin([f"age_{i}" for i in range(age_from, age_to)])]].sum(axis=1)
    df[f"grunnkrets_age_{age_from}-{age_to-1}_distribution"] = df[f"grunnkrets_age_{age_from}-{age_to-1}"]/df["grunnkrets_population"]

for name in ["district_name", "municipality_name"]:
    for j, age in enumerate(ages):
        prefix = name.split("_")[0]
        if j == 0:
            pass
            # columns = [f"age_{i}" for i in range(age)]
            # columns.append(name)
            # df[f"{prefix}_age_0-{ages[j]-1}"] = df[df.columns[df.columns.isin(columns)]].groupby(name).transform("sum").sum(axis=1)
        else:
            columns = [f"age_{i}" for i in range(ages[j-1], age)]
            columns.append(name)
            df[f"{prefix}_age_{ages[j-1]}-{ages[j]-1}"] = df[df.columns[df.columns.isin(columns)]].groupby(name).transform("sum").sum(axis=1)
            df[f"{prefix}_age_{ages[j-1]}-{ages[j]-1}_distribution"] = df[f"{prefix}_age_{ages[j-1]}-{ages[j]-1}"]/df[f"{prefix}_population"]
# There is currently an error with this, so it is currently discarded
# for name in ["grunnkrets", "district", "municipality"]:
#     print(df.columns.tolist())
#     df[f"{name}_pop_number"] = np.sum([df[f"{name}_age_{ages[j-1]}-{ages[j]-1}"]*(j - len(ages)/2)/(1-len(ages)/2) for j in range(1, len(ages))])/df[f"{name}_population"] 
df["grunnkrets_density"] = df.grunnkrets_population/df.area_km2
df["district_density"] = df.district_population/df.district_area
df["municipality_density"] = df.municipality_population/df.municipality_area

df = df.drop(columns=[f"age_{i}" for i in range(91)])
df = df.drop(columns=["year_y"])
df = df.rename(columns={"year_x": "year"})
# df.T

In [ ]:
df.to_csv("../../own_data/grunnkrets_norway_large.csv", index=False)

In [ ]:
all_data = pd.read_csv("../../own_data/all_merged.csv")
all_data_large = pd.read_csv("../../own_data/grunnkrets_norway_large.csv")

ages = [0,15, 35,65,91]

cols = []
for name in ["district_age", "municipality_age"]:
    for j in range(1, len(ages)):  
        age_from = ages[j-1]
        age_to = ages[j]
        col_name = f"{name}_{age_from}-{age_to-1}"
        # cols.append(col_name)
        col_name = col_name + "_distribution"
        cols.append(col_name)

        col_mean, col_std = all_data_large[col_name].mean(), all_data_large[col_name].std()
        
        # all_data_large[all_data_large[col_name]> col_mean + 3*col_std] = all_data_large[all_data_large[col_name]> col_mean + 3*col_std].apply(lambda x: col_mean +np.random.uniform(-1,1)* col_std)

# print(all_data_large['district_age_0-2'])
cols.extend(['grunnkrets_id', 
        'grunnkrets_population',
        'district_population', 
        'municipality_population', 
        'district_area', 
        'municipality_area', 
        'municipality_density', 
        'district_density', 
])
all_data_large = all_data_large[cols]
all_data.set_index(["dataset", "range_index"], inplace=True)
all_data['copy_index'] = all_data.index
all_data = all_data.merge(all_data_large, how="left", on='grunnkrets_id')
all_data.index = pd.MultiIndex.from_tuples(
    all_data['copy_index'], 
    names=['dataset', 'range_index'])
all_data.drop(columns=['age_'+str(i) for i in range(91)], inplace=True)


In [ ]:
all_data = all_data.groupby(["lv1", "district_name"]) \
    .apply(lambda gdf: 
        gdf.assign(
            lv1_population_district_div_count_stores=lambda gdf:  gdf["district_population"].mean() / gdf["store_id"].count() 
        )
    ).droplevel(0)

all_data.index = pd.MultiIndex.from_tuples(
    all_data['copy_index'], 
    names=['dataset', 'range_index'])

all_data = all_data.groupby(["lv2", "district_name"]) \
    .apply(lambda gdf: 
        gdf.assign(
            lv2_population_district_div_count_stores=lambda gdf:  gdf["district_population"].mean() / gdf["store_id"].count() 
        )
    ).droplevel(0)

all_data.index = pd.MultiIndex.from_tuples(
    all_data['copy_index'], 
    names=['dataset', 'range_index'])


all_data = all_data.groupby(["lv3", "district_name"]) \
    .apply(lambda gdf: 
        gdf.assign(
            lv3_population_district_div_count_stores=lambda gdf:  gdf["district_population"].mean() / gdf["store_id"].count() 
        )
    ).droplevel(0)

all_data.index = pd.MultiIndex.from_tuples(
    all_data['copy_index'], 
    names=['dataset', 'range_index'])

all_data = all_data.groupby(["lv4", "district_name"]) \
    .apply(lambda gdf: 
        gdf.assign(
            lv4_population_district_div_count_stores=lambda gdf:  gdf["district_population"].mean() / gdf["store_id"].count() 
        )
    ).droplevel(0)

all_data.index = pd.MultiIndex.from_tuples(
    all_data['copy_index'], 
    names=['dataset', 'range_index'])

all_data = all_data.groupby(["lv1", "municipality_name"]) \
    .apply(lambda gdf: 
        gdf.assign(
            lv1_population_municipality_div_count_stores=lambda gdf:  gdf["municipality_population"].mean() / gdf["store_id"].count() 
        )
    ).droplevel(0)

all_data.index = pd.MultiIndex.from_tuples(
    all_data['copy_index'], 
    names=['dataset', 'range_index'])

all_data = all_data.groupby(["lv2", "municipality_name"]) \
    .apply(lambda gdf: 
        gdf.assign(
            lv2_population_municipality_div_count_stores=lambda gdf:  gdf["municipality_population"].mean() / gdf["store_id"].count() 
        )
    ).droplevel(0)

all_data.index = pd.MultiIndex.from_tuples(
    all_data['copy_index'], 
    names=['dataset', 'range_index'])

all_data = all_data.groupby(["lv3", "municipality_name"]) \
    .apply(lambda gdf: 
        gdf.assign(
            lv3_population_municipality_div_count_stores=lambda gdf:  gdf["municipality_population"].mean() / gdf["store_id"].count() 
        )
    ).droplevel(0)

all_data.index = pd.MultiIndex.from_tuples(
    all_data['copy_index'], 
    names=['dataset', 'range_index'])

all_data = all_data.groupby(["lv4", "municipality_name"]) \
    .apply(lambda gdf: 
        gdf.assign(
            lv4_population_municipality_div_count_stores=lambda gdf:  gdf["municipality_population"].mean() / gdf["store_id"].count() 
        )
    ).droplevel(0)

all_data.index = pd.MultiIndex.from_tuples(
    all_data['copy_index'], 
    names=['dataset', 'range_index'])

all_data.drop(columns='copy_index', inplace=True)


In [ ]:
all_data.to_csv("../../own_data/all_with_stores_pop.csv")